In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import random
import json
import zipfile
import io
import math
import time
from collections import deque, defaultdict
from copy import deepcopy
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict
import ast

# ============================================================================
# 1. CORE CLASSES (Game Rules & AI)
# ============================================================================

@dataclass
class Move:
    start: Tuple[int, int]
    end: Tuple[int, int]
    piece: str
    captured: Optional[str] = None
    promotion: Optional[str] = None
    is_check: bool = False
    is_checkmate: bool = False
    score: float = 0.0
    
    def __hash__(self):
        # FIXED: Includes captured/promotion to prevent hash collisions
        return hash((self.start, self.end, self.piece, self.captured, self.promotion))
    
    def __eq__(self, other):
        # FIXED: Includes captured/promotion to ensure distinct moves are treated differently
        return (self.start == other.start and self.end == other.end and 
                self.piece == other.piece and self.captured == other.captured and 
                self.promotion == other.promotion)
    
    def to_notation(self):
        cols = 'abcde'
        s = f"{cols[self.start[1]]}{5-self.start[0]}"
        e = f"{cols[self.end[1]]}{5-self.end[0]}"
        notation = f"{s}{e}"
        if self.promotion: notation += f"={self.promotion.upper()}"
        if self.is_checkmate: notation += "#"
        elif self.is_check: notation += "+"
        return notation

class Minichess:
    PIECE_VALUES = {'P': 100, 'N': 320, 'B': 330, 'R': 500, 'Q': 900, 'K': 20000,
                    'p': -100, 'n': -320, 'b': -330, 'r': -500, 'q': -900, 'k': -20000}
    
    PST = {
        'P': [[0, 0, 0, 0, 0], [80, 80, 80, 80, 80], [50, 50, 60, 50, 50], [10, 10, 20, 10, 10], [5, 5, 10, 5, 5]],
        'N': [[-50,-40,-30,-40,-50], [-40,-20,0,-20,-40], [-30,5,15,5,-30], [-40,-20,0,-20,-40], [-50,-40,-30,-40,-50]],
        'B': [[-20,-10,-10,-10,-20], [-10,5,0,5,-10], [-10,10,10,10,-10], [-10,5,10,5,-10], [-20,-10,-10,-10,-20]],
        'R': [[0,0,0,0,0], [5,10,10,10,5], [-5,0,0,0,-5], [-5,0,0,0,-5], [0,0,0,0,0]],
        'Q': [[-20,-10,-10,-10,-20], [-10,0,0,0,-10], [-10,0,5,0,-10], [-10,0,0,0,-10], [-20,-10,-10,-10,-20]],
        'K': [[-30,-40,-40,-40,-30], [-30,-40,-40,-40,-30], [-30,-40,-40,-40,-30], [-20,-30,-30,-30,-20], [-10,-20,-20,-20,-10]]
    }
    
    OPENING_BOOK = {
        '(("k", "q", "b", "n", "r"), ("p", "p", "p", "p", "p"), (".", ".", ".", ".", "."), ("P", "P", "P", "P", "P"), ("K", "Q", "B", "N", "R"))': [
            ((3, 2), (2, 2)), ((3, 1), (2, 1))
        ]
    }
    
    def __init__(self):
        self.board_size = 5
        self.transposition_table = {}
        self.killer_moves = [[] for _ in range(20)]
        self.history_table = defaultdict(int)
        self.reset()
    
    def reset(self):
        self.board = np.array([
            ['k', 'q', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'p', 'p'],
            ['.', '.', '.', '.', '.'],
            ['P', 'P', 'P', 'P', 'P'],
            ['K', 'Q', 'B', 'N', 'R']
        ])
        self.current_player = 1
        self.game_over = False
        self.winner = None
        self.move_history = []
        self.move_count = 0
        return self.get_state()
    
    def get_state(self):
        return tuple(tuple(str(c) for c in row) for row in self.board)
    
    def copy(self):
        new_game = Minichess()
        new_game.board = self.board.copy()
        new_game.current_player = self.current_player
        new_game.game_over = self.game_over
        new_game.winner = self.winner
        new_game.move_history = self.move_history.copy()
        new_game.move_count = self.move_count
        new_game.transposition_table = self.transposition_table
        new_game.killer_moves = self.killer_moves
        new_game.history_table = self.history_table
        return new_game
    
    def is_white_piece(self, piece): return piece.isupper() and piece != '.'
    def is_black_piece(self, piece): return piece.islower() and piece != '.'
    def is_enemy(self, piece, player): return self.is_black_piece(piece) if player == 1 else self.is_white_piece(piece)
    def is_friendly(self, piece, player): return self.is_white_piece(piece) if player == 1 else self.is_black_piece(piece)
    
    def order_moves(self, moves, ply=0):
        for move in moves:
            score = 0
            if move.captured:
                victim = abs(self.PIECE_VALUES.get(move.captured, 0))
                attacker = abs(self.PIECE_VALUES.get(move.piece, 0))
                score += 10000 + victim - attacker / 100
            if ply < len(self.killer_moves) and move in self.killer_moves[ply]: score += 9000
            score += self.history_table.get((move.start, move.end), 0)
            if move.promotion: score += 8000
            if move.is_check: score += 5000
            move.score = score
        return sorted(moves, key=lambda m: m.score, reverse=True)
    
    def get_piece_moves(self, row, col, check_legal=True):
        piece = self.board[row, col]
        if piece == '.' or not self.is_friendly(piece, self.current_player): return []
        
        moves = []
        pt = piece.upper()
        if pt == 'P': moves = self._get_pawn_moves(row, col)
        elif pt == 'N': moves = self._get_knight_moves(row, col)
        elif pt == 'B': moves = self._get_sliding_moves(row, col, [(-1,-1),(-1,1),(1,-1),(1,1)])
        elif pt == 'R': moves = self._get_sliding_moves(row, col, [(-1,0),(1,0),(0,-1),(0,1)])
        elif pt == 'Q': moves = self._get_sliding_moves(row, col, [(-1,-1),(-1,1),(1,-1),(1,1),(-1,0),(1,0),(0,-1),(0,1)])
        elif pt == 'K': moves = self._get_king_moves(row, col)
        
        if check_legal:
            legal_moves = []
            for m in moves:
                tg = self.copy()
                tg._make_move_internal(m)
                if not tg._is_in_check(self.current_player): legal_moves.append(m)
            return legal_moves
        return moves

    def _get_pawn_moves(self, row, col):
        moves = []
        piece = self.board[row, col]
        dr = -1 if self.current_player == 1 else 1
        promo_row = 0 if self.current_player == 1 else 4
        
        # Forward
        nr = row + dr
        if 0 <= nr < 5 and self.board[nr, col] == '.':
            if nr == promo_row:
                for p in ['Q', 'R', 'B', 'N']: moves.append(Move((row, col), (nr, col), piece, promotion=p))
            else: moves.append(Move((row, col), (nr, col), piece))
        
        # Capture
        for dc in [-1, 1]:
            nc = col + dc
            if 0 <= nr < 5 and 0 <= nc < 5:
                t = self.board[nr, nc]
                if t != '.' and self.is_enemy(t, self.current_player):
                    if nr == promo_row:
                        for p in ['Q', 'R', 'B', 'N']: moves.append(Move((row, col), (nr, nc), piece, captured=t, promotion=p))
                    else: moves.append(Move((row, col), (nr, nc), piece, captured=t))
        return moves

    def _get_knight_moves(self, r, c):
        moves = []
        p = self.board[r, c]
        for dr, dc in [(-2,-1),(-2,1),(-1,-2),(-1,2),(1,-2),(1,2),(2,-1),(2,1)]:
            nr, nc = r+dr, c+dc
            if 0<=nr<5 and 0<=nc<5:
                t = self.board[nr, nc]
                if t == '.' or self.is_enemy(t, self.current_player):
                    moves.append(Move((r,c), (nr,nc), p, captured=(t if t!='.' else None)))
        return moves

    def _get_sliding_moves(self, r, c, dirs):
        moves = []
        p = self.board[r, c]
        for dr, dc in dirs:
            for i in range(1, 5):
                nr, nc = r + dr*i, c + dc*i
                if not (0<=nr<5 and 0<=nc<5): break
                t = self.board[nr, nc]
                if t == '.': moves.append(Move((r,c), (nr,nc), p))
                elif self.is_enemy(t, self.current_player):
                    moves.append(Move((r,c), (nr,nc), p, captured=t))
                    break
                else: break
        return moves

    def _get_king_moves(self, r, c):
        moves = []
        p = self.board[r, c]
        for dr in [-1,0,1]:
            for dc in [-1,0,1]:
                if dr==0 and dc==0: continue
                nr, nc = r+dr, c+dc
                if 0<=nr<5 and 0<=nc<5:
                    t = self.board[nr, nc]
                    if t=='.' or self.is_enemy(t, self.current_player):
                        moves.append(Move((r,c), (nr,nc), p, captured=(t if t!='.' else None)))
        return moves

    def get_all_valid_moves(self):
        state = self.get_state()
        if state in self.OPENING_BOOK and self.move_count < 3:
            bm = []
            for s, e in self.OPENING_BOOK[state]:
                p = self.board[s[0], s[1]]
                c = self.board[e[0], e[1]] if self.board[e[0], e[1]] != '.' else None
                bm.append(Move(s, e, p, captured=c))
            if bm: return bm
        
        all_moves = []
        for r in range(5):
            for c in range(5):
                if self.board[r, c] != '.' and self.is_friendly(self.board[r, c], self.current_player):
                    all_moves.extend(self.get_piece_moves(r, c))
        return self.order_moves(all_moves, self.move_count)

    def _find_king(self, player):
        k = 'K' if player == 1 else 'k'
        for r in range(5):
            for c in range(5):
                if self.board[r,c] == k: return (r,c)
        return None

    def _is_square_attacked(self, r, c, by_p):
        orig = self.current_player
        self.current_player = by_p
        for row in range(5):
            for col in range(5):
                p = self.board[row, col]
                if p != '.' and self.is_friendly(p, by_p):
                    for m in self.get_piece_moves(row, col, check_legal=False):
                        if m.end == (r, c):
                            self.current_player = orig
                            return True
        self.current_player = orig
        return False

    def _is_in_check(self, p):
        kp = self._find_king(p)
        return kp and self._is_square_attacked(kp[0], kp[1], 3-p)

    def _make_move_internal(self, m):
        sr, sc = m.start
        er, ec = m.end
        p = m.promotion if m.promotion else self.board[sr, sc]
        if m.promotion and self.current_player == 2: p = p.lower()
        self.board[er, ec] = p
        self.board[sr, sc] = '.'

    def make_move(self, m: Move):
        if self.game_over: return self.get_state(), 0, True
        
        reward = 0
        if m.captured: reward = abs(self.PIECE_VALUES.get(m.captured, 0)) / 100
        if m.promotion: reward += 5
        
        self._make_move_internal(m)
        self.move_history.append(m)
        self.move_count += 1
        self.history_table[(m.start, m.end)] += 1
        self.current_player = 3 - self.current_player
        
        om = self.get_all_valid_moves()
        chk = self._is_in_check(self.current_player)
        
        if not om:
            self.game_over = True
            if chk:
                self.winner = 3 - self.current_player
                reward = 100
                m.is_checkmate = True
            else:
                self.winner = 0
                reward = 0
        elif chk:
            m.is_check = True
            reward += 1
            
        if self.move_count >= 100:
            self.game_over = True
            self.winner = 0
            
        return self.get_state(), reward, self.game_over

    def evaluate_position(self, p):
        if self.winner == p: return 100000
        if self.winner == (3-p): return -100000
        if self.winner == 0: return 0
        
        score = 0
        mat = 0
        pos = 0
        for r in range(5):
            for c in range(5):
                pc = self.board[r, c]
                if pc == '.': continue
                mine = self.is_friendly(pc, p)
                mul = 1 if mine else -1
                mat += mul * abs(self.PIECE_VALUES.get(pc, 0))
                pt = pc.upper()
                if pt in self.PST:
                    val = self.PST[pt][r][c] if pc.isupper() else self.PST[pt][4-r][c]
                    pos += mul * val
        
        score = mat + pos
        
        orig = self.current_player
        self.current_player = p
        my_m = len(self.get_all_valid_moves())
        self.current_player = 3-p
        op_m = len(self.get_all_valid_moves())
        self.current_player = orig
        
        score += (my_m - op_m) * 10
        if self._is_in_check(p): score -= 50
        if self._is_in_check(3-p): score += 50
        return score

class MCTSNode:
    def __init__(self, gs, parent=None, move=None, prior=1.0):
        self.game_state = gs
        self.parent = parent
        self.move = move
        self.prior = prior
        self.gumbel = np.random.gumbel(0, 1)
        self.children = {}
        self.visit_count = 0
        self.value_sum = 0.0
        self.is_expanded = False
    
    def value(self): return self.value_sum / self.visit_count if self.visit_count > 0 else 0
    
    def ucb_score(self, pv, c_puct=1.4, c_visit=50, c_scale=1.0):
        if self.visit_count == 0: q = 0
        else: q = (self.value() + 1) / 2
        u = c_puct * self.prior * math.sqrt(pv) / (1 + self.visit_count)
        g = self.gumbel + math.log(self.prior) + c_scale * q * c_visit
        return q + u + g / (1 + pv)
    
    def select_child(self, c_puct=1.4):
        if not self.children: return None
        cands = list(self.children.values())
        if len(cands) <= 1: return cands[0] if cands else None
        cands.sort(key=lambda c: c.ucb_score(self.visit_count, c_puct), reverse=True)
        if self.visit_count > 10: cands = cands[:max(1, len(cands)//2)]
        return max(cands, key=lambda c: c.ucb_score(self.visit_count, c_puct))
    
    def expand(self, game, priors):
        vm = game.get_all_valid_moves()
        if not vm: return
        tp = sum(priors.values()) or len(vm)
        if self.parent is None:
            dn = np.random.dirichlet([0.3] * len(vm))
        
        for idx, m in enumerate(vm):
            p = priors.get(m, 1.0) / tp
            if self.parent is None: p = 0.75*p + 0.25*dn[idx]
            cg = game.copy()
            cg.make_move(m)
            self.children[m] = MCTSNode(cg, self, m, p)
        self.is_expanded = True
    
    def backup(self, v):
        self.visit_count += 1
        self.value_sum += v
        if self.parent: self.parent.backup(-v)

class Agent:
    def __init__(self, pid, lr=0.5, gamma=0.99):
        self.player_id = pid
        self.lr = lr
        self.gamma = gamma
        self.epsilon = 1.0
        self.epsilon_decay = 0.92
        self.epsilon_min = 0.05
        self.mcts_simulations = 200
        self.minimax_depth = 4
        self.replay_buffer = deque(maxlen=10000)
        self.policy_table = defaultdict(lambda: defaultdict(float))
        self.value_table = {}
        self.wins = 0
        self.losses = 0
        self.draws = 0
        self.training_steps = 0
        self.c_puct = 1.4

    def get_policy_priors(self, game):
        state = game.get_state()
        moves = game.get_all_valid_moves()
        priors = {}
        for m in moves:
            if state in self.policy_table and m in self.policy_table[state]:
                p = self.policy_table[state][m]
            else:
                p = 1.0
                if m.captured: p += (abs(Minichess.PIECE_VALUES.get(m.captured, 0)) - abs(Minichess.PIECE_VALUES.get(m.piece, 0))/100)/100
                if m.promotion == 'Q': p += 5.0
                elif m.promotion: p += 3.0
                if m.is_checkmate: p += 10.0
                elif m.is_check: p += 2.0
                if 1 <= m.end[0] <= 3 and 1 <= m.end[1] <= 3: p += 0.5
            priors[m] = max(p, 0.01)
        return priors

    def mcts_search(self, game, sims):
        root = MCTSNode(game.copy())
        es = min(sims, 50 + self.training_steps // 10)
        for _ in range(es):
            node = root
            sg = game.copy()
            while node.is_expanded and node.children:
                node = node.select_child(self.c_puct)
                if not node: break
                sg.make_move(node.move)
            if node and not sg.game_over:
                pp = self.get_policy_priors(sg)
                node.expand(sg, pp)
            if node:
                val = self._evaluate_leaf(sg)
                node.backup(val)
        return root

    def _evaluate_leaf(self, game):
        if game.game_over:
            if game.winner == self.player_id: return 1.0
            elif game.winner == (3-self.player_id): return -1.0
            return 0.0
        state = game.get_state()
        if state in self.value_table: return self.value_table[state]
        if any(m.captured or m.is_check for m in game.get_all_valid_moves()[:3]):
            s = self._quiescence_search(game, 3, -float('inf'), float('inf'), True)
        else:
            s = self._iterative_deepening_minimax(game, self.minimax_depth)
        v = np.tanh(s/500)
        self.value_table[state] = v
        return v

    def _quiescence_search(self, g, d, a, b, maxing):
        sp = g.evaluate_position(self.player_id)
        if d==0: return sp
        if maxing:
            if sp >= b: return b
            a = max(a, sp)
        else:
            if sp <= a: return a
            b = min(b, sp)
        
        moves = [m for m in g.get_all_valid_moves() if m.captured]
        if not moves: return sp
        
        if maxing:
            me = sp
            for m in moves[:5]:
                cg = g.copy()
                cg.make_move(m)
                v = self._quiescence_search(cg, d-1, a, b, False)
                me = max(me, v)
                a = max(a, v)
                if b <= a: break
            return me
        else:
            me = sp
            for m in moves[:5]:
                cg = g.copy()
                cg.make_move(m)
                v = self._quiescence_search(cg, d-1, a, b, True)
                me = min(me, v)
                b = min(b, v)
                if b <= a: break
            return me

    def _iterative_deepening_minimax(self, g, md):
        s = 0
        for d in range(1, md+1):
            try: s = self._minimax(g, d, -float('inf'), float('inf'), True)
            except: pass
        return s

    def _minimax(self, g, d, a, b, maxing):
        if d==0 or g.game_over: return g.evaluate_position(self.player_id)
        m = g.get_all_valid_moves()
        if not m: return g.evaluate_position(self.player_id)
        if len(m)>10: m = m[:max(8, 15-d)]
        
        if maxing:
            me = -float('inf')
            for mv in m:
                cg = g.copy()
                cg.make_move(mv)
                ev = self._minimax(cg, d-1, a, b, False)
                me = max(me, ev)
                a = max(a, ev)
                if b<=a: 
                    if mv not in g.killer_moves[g.move_count]: g.killer_moves[g.move_count].append(mv)
                    break
            return me
        else:
            me = float('inf')
            for mv in m:
                cg = g.copy()
                cg.make_move(mv)
                ev = self._minimax(cg, d-1, a, b, True)
                me = min(me, ev)
                b = min(b, ev)
                if b<=a: break
            return me

    def choose_action(self, game, training=True):
        m = game.get_all_valid_moves()
        if not m: return None
        if training and random.random() < self.epsilon: return random.choice(m)
        root = self.mcts_search(game, self.mcts_simulations)
        if not root.children: return random.choice(m)
        
        if training and game.move_count < 10:
            v = np.array([c.visit_count for c in root.children.values()])
            p = (v ** 1.0); p /= p.sum()
            bm = np.random.choice(list(root.children.keys()), p=p)
        else:
            bm = max(root.children.items(), key=lambda x: x[1].visit_count)[0]
        
        s = game.get_state()
        tv = sum(c.visit_count for c in root.children.values())
        for mv, c in root.children.items(): self.policy_table[s][mv] = c.visit_count / tv
        self.replay_buffer.append((s, bm, root.value()))
        return bm

    def update_from_game(self, hist, res):
        self.training_steps += 1
        for s, m, p in hist:
            if p != self.player_id: continue
            rew = 1.0 if res == self.player_id else (0.0 if res == 0 else -1.0)
            cur = self.policy_table[s][m]
            self.policy_table[s][m] = cur + self.lr * (rew - cur)
        
        if len(self.replay_buffer) > 100:
            bat = random.sample(list(self.replay_buffer), min(32, len(self.replay_buffer)))
            for s, m, v in bat:
                if s in self.policy_table and m in self.policy_table[s]:
                    ov = self.policy_table[s][m]
                    self.policy_table[s][m] = ov + 0.1 * self.lr * (v - ov)
    
    def decay_epsilon(self): self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    def reset_stats(self): self.wins = 0; self.losses = 0; self.draws = 0

def play_game(env, a1, a2, training=True):
    env.reset()
    gh = []
    ags = {1: a1, 2: a2}
    mc = 0
    while not env.game_over and mc < 100:
        cp = env.current_player
        ag = ags[cp]
        s = env.get_state()
        m = ag.choose_action(env, training)
        if m is None: break
        gh.append((s, m, cp))
        env.make_move(m)
        mc += 1
    
    if env.winner == 1:
        a1.wins+=1; a2.losses+=1
        if training: a1.update_from_game(gh, 1); a2.update_from_game(gh, 1)
    elif env.winner == 2:
        a2.wins+=1; a1.losses+=1
        if training: a1.update_from_game(gh, 2); a2.update_from_game(gh, 2)
    else:
        a1.draws+=1; a2.draws+=1
        if training: a1.update_from_game(gh, 0); a2.update_from_game(gh, 0)
    return env.winner

In [ ]:
# ============================================================================
# 2. TRAINING & SERIALIZATION (Strictly Matches Streamlit App)
# ============================================================================

# !!! CRITICAL !!! 
# This matches the "deserialize_move" function in your Streamlit app exactly.
# Keys: "s" (start), "e" (end), "p" (piece), "c" (captured), "pr" (promotion)
def serialize_move(move):
    return {
        "s": [int(x) for x in move.start],
        "e": [int(x) for x in move.end],
        "p": str(move.piece),
        "c": str(move.captured) if move.captured else None,
        "pr": str(move.promotion) if move.promotion else None
    }

def save_kaggle_brain(agent1, agent2, filename="grandmaster_minichess.zip"):
    # Create the config dictionary your app expects
    config = {
        "lr1": agent1.lr,
        "mcts_sims1": agent1.mcts_simulations,
        "minimax_depth1": agent1.minimax_depth,
        "lr2": agent2.lr,
        "mcts_sims2": agent2.mcts_simulations,
        "minimax_depth2": agent2.minimax_depth,
        "training_history": {
            "source": "Kaggle GPU Training",
            "final_epsilon": agent1.epsilon,
            "episode": list(range(1, agent1.training_steps + 1)) # Simple history placeholder
        }
    }

    # Helper to clean up the policy table for JSON
    def serialize_agent(agent, role_name):
        clean_policy = {}
        # Copy to avoid changing the active brain during save
        current_policies = agent.policy_table.copy()
        
        for state, moves in current_policies.items():
            try:
                # Convert tuple state to string (Matches ast.literal_eval in your app)
                state_str = str(state)
                clean_policy[state_str] = {}
                
                for move, value in moves.items():
                    # Serialize using the strict function above
                    move_json_str = json.dumps(serialize_move(move))
                    clean_policy[state_str][move_json_str] = float(value)
            except Exception:
                continue
        
        return {
            "metadata": {"role": role_name, "version": "3.1_KAGGLE"},
            "policy_table": clean_policy,
            "epsilon": float(agent.epsilon),
            "wins": int(agent.wins),
            "losses": int(agent.losses),
            "draws": int(agent.draws),
            "mcts_sims": int(agent.mcts_simulations),
            "training_steps": int(agent.training_steps)
        }
    
    # Generate the JSON data
    data1 = serialize_agent(agent1, "White")
    data2 = serialize_agent(agent2, "Black")
    
    # Write to physical file
    print(f"💾 Saving {len(data1['policy_table'])} policies to {filename}...")
    with zipfile.ZipFile(filename, "w", zipfile.ZIP_DEFLATED) as zf:
        zf.writestr("agent1.json", json.dumps(data1))
        zf.writestr("agent2.json", json.dumps(data2))
        zf.writestr("config.json", json.dumps(config))
    print(f"✅ Save Complete! Download '{filename}' and upload to your Streamlit app.")

# --- THE TRAINING LOOP ---
def train_grandmaster():
    # ADJUST THESE FOR KAGGLE POWER
    EPISODES = 30    # Try 2000 or 5000 if you have time!
    MCTS_SIMS = 100      # Higher = Smarter logic
    DEPTH = 5            # Deeper calculation
    CHECKPOINT_FREQ = 1 
    
    env = Minichess()
    # Initialize agents
    agent1 = Agent(1, lr=0.3)
    agent2 = Agent(2, lr=0.3)
    
    # Set compute budget
    agent1.mcts_simulations = MCTS_SIMS
    agent2.mcts_simulations = MCTS_SIMS
    agent1.minimax_depth = DEPTH
    agent2.minimax_depth = DEPTH
    
    start_time = time.time()
    print(f"⚔️  Beginning Grandmaster Training ({EPISODES} Episodes)...")
    
    for i in range(1, EPISODES + 1):
        # Play game
        play_game(env, agent1, agent2, training=True)
        
        # Decay epsilon
        agent1.decay_epsilon()
        agent2.decay_epsilon()
        
        # Log Progress
        if i % 10 == 0:
            elapsed = time.time() - start_time
            rate = i / elapsed
            print(f"Game {i:04d} | Policies: {len(agent1.policy_table):,} | "
                  f"W: {agent1.wins} L: {agent1.losses} | ε: {agent1.epsilon:.3f} | {rate:.1f} games/s")
            
        # Intermediate Save (Safety)
        if i % CHECKPOINT_FREQ == 0:
            save_kaggle_brain(agent1, agent2, filename=f"checkpoint_{i}.zip")

    # Final Save
    print("\n🏆 Training Finished!")
    save_kaggle_brain(agent1, agent2, filename="grandmaster_minichess.zip")

# Start Training
if __name__ == "__main__":
    train_grandmaster()

⚔️  Beginning Grandmaster Training (30 Episodes)...
💾 Saving 49 policies to checkpoint_1.zip...
✅ Save Complete! Download 'checkpoint_1.zip' and upload to your Streamlit app.
💾 Saving 98 policies to checkpoint_2.zip...
✅ Save Complete! Download 'checkpoint_2.zip' and upload to your Streamlit app.
💾 Saving 146 policies to checkpoint_3.zip...
✅ Save Complete! Download 'checkpoint_3.zip' and upload to your Streamlit app.
💾 Saving 173 policies to checkpoint_4.zip...
✅ Save Complete! Download 'checkpoint_4.zip' and upload to your Streamlit app.
💾 Saving 220 policies to checkpoint_5.zip...
✅ Save Complete! Download 'checkpoint_5.zip' and upload to your Streamlit app.
💾 Saving 230 policies to checkpoint_6.zip...
✅ Save Complete! Download 'checkpoint_6.zip' and upload to your Streamlit app.
💾 Saving 242 policies to checkpoint_7.zip...
✅ Save Complete! Download 'checkpoint_7.zip' and upload to your Streamlit app.
💾 Saving 290 policies to checkpoint_8.zip...
✅ Save Complete! Download 'checkpoint